## Quarterfinals

In [47]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

Load data

In [50]:
# Load all football match data since 1872
data = pd.read_csv('international_matches.csv')
data.head()

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,shoot_out,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score
0,1993-08-08,Bolivia,Uruguay,South America,South America,59,22,0,0,3,...,No,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1993-08-08,Brazil,Mexico,South America,North America,8,14,0,0,1,...,No,Draw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1993-08-08,Ecuador,Venezuela,South America,South America,35,94,0,0,5,...,No,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1993-08-08,Guinea,Sierra Leone,Africa,Africa,65,86,0,0,1,...,No,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1993-08-08,Paraguay,Argentina,South America,South America,67,5,0,0,1,...,No,Lose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23921 entries, 0 to 23920
Data columns (total 25 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   date                           23921 non-null  object 
 1   home_team                      23921 non-null  object 
 2   away_team                      23921 non-null  object 
 3   home_team_continent            23921 non-null  object 
 4   away_team_continent            23921 non-null  object 
 5   home_team_fifa_rank            23921 non-null  int64  
 6   away_team_fifa_rank            23921 non-null  int64  
 7   home_team_total_fifa_points    23921 non-null  int64  
 8   away_team_total_fifa_points    23921 non-null  int64  
 9   home_team_score                23921 non-null  int64  
 10  away_team_score                23921 non-null  int64  
 11  tournament                     23921 non-null  object 
 12  city                           23921 non-null 

In [52]:
# Format date
data['date'] = pd.to_datetime(data['date'])

In [53]:
data.columns

Index(['date', 'home_team', 'away_team', 'home_team_continent',
       'away_team_continent', 'home_team_fifa_rank', 'away_team_fifa_rank',
       'home_team_total_fifa_points', 'away_team_total_fifa_points',
       'home_team_score', 'away_team_score', 'tournament', 'city', 'country',
       'neutral_location', 'shoot_out', 'home_team_result',
       'home_team_goalkeeper_score', 'away_team_goalkeeper_score',
       'home_team_mean_defense_score', 'home_team_mean_offense_score',
       'home_team_mean_midfield_score', 'away_team_mean_defense_score',
       'away_team_mean_offense_score', 'away_team_mean_midfield_score'],
      dtype='object')

In [54]:
# Keep only relevent columns
data.drop(['home_team_continent',
       'away_team_continent','tournament', 'city', 'country',
       'neutral_location'], axis = 1)

,date,home_team,away_team,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,away_team_score,shoot_out,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score
0,1993-08-08,Bolivia,Uruguay,59,22,0,0,3,1,No,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1993-08-08,Brazil,Mexico,8,14,0,0,1,1,No,Draw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1993-08-08,Ecuador,Venezuela,35,94,0,0,5,0,No,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1993-08-08,Guinea,Sierra Leone,65,86,0,0,1,0,No,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1993-08-08,Paraguay,Argentina,67,5,0,0,1,3,No,Lose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23916,2022-06-14,Moldova,Andorra,180,153,932,1040,2,1,No,Win,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23917,2022-06-14,Liechtenstein,Latvia,192,135,895,1105,0,2,No,Lose,NaN,65.0,NaN,NaN,NaN,NaN,NaN,NaN
23918,2022-06-14,Chile,Ghana,28,60,1526,1387,0,0,Yes,Lose,79.0,74.0,75.5,76.7,78.2,75.5,76.0,78.2
23919,2022-06-14,Japan,Tunisia,23,35,1553,1499,0,3,No,Lose,73.0,NaN,75.2,75.0,77.5,70.8,72.3,74.0


In [55]:
fifa_rank = data[['date', 'home_team', 'away_team', 'home_team_fifa_rank', 'away_team_fifa_rank', 'away_team_total_fifa_points', 'home_team_total_fifa_points',
        ]]
home = data[['date', 'home_team', 'home_team_fifa_rank', 'home_team_total_fifa_points']].rename(columns={'home_team': 'team', 'home_team_fifa_rank': 'rank','home_team_total_fifa_points': 'rank_points'})
away = data[['date', 'away_team', 'away_team_fifa_rank', 'away_team_total_fifa_points']].rename(columns={'away_team': 'team', 'away_team_fifa_rank': 'rank', 
                                                                                                          'away_team_total_fifa_points':'rank_points'})
fifa_rank = home.append(away)
fifa_rank = fifa_rank.sort_values(['team', 'date'], ascending = [True, False])
fifa_rank['row_number'] = fifa_rank.groupby('team').cumcount()+1
# Look at 10 top teams
fifa_rank_top = fifa_rank[fifa_rank['row_number']==1].drop('row_number', axis = 1).nsmallest(10, 'rank')

In [56]:
fifa_rank_top

,date,team,rank,rank_points
23760,2022-06-06,Brazil,1,1832
23909,2022-06-14,Belgium,2,1827
23885,2022-06-13,France,3,1789
23741,2022-06-05,Argentina,4,1765
23906,2022-06-14,England,5,1761
23907,2022-06-14,Italy,6,1723
23866,2022-06-12,Spain,7,1709
23867,2022-06-12,Portugal,8,1674
23903,2022-06-14,Mexico,9,1658
23908,2022-06-14,Netherlands,10,1658


In [57]:
# Find columns with null values
col_with_nulls = [col for col in data.columns if data[col].isnull().any()]
for col in col_with_nulls:
    print(col)

home_team_goalkeeper_score
away_team_goalkeeper_score
home_team_mean_defense_score
home_team_mean_offense_score
home_team_mean_midfield_score
away_team_mean_defense_score
away_team_mean_offense_score
away_team_mean_midfield_score


In [58]:
# Teams in quarterfinals
quarters_teams = ['Brazil', 'Croatia', 'Arentina', 'Netherlands', 'Englad', 'France', 'Morocco', 'Portugal']

In [59]:
# Fill null values with mean
for i in quarters_teams:
    for j in col_with_nulls:
        data[j].fillna(data[data[j[0:9]]==i][j].mean(), inplace = True)

In [60]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23921 entries, 0 to 23920
Data columns (total 25 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   date                           23921 non-null  datetime64[ns]
 1   home_team                      23921 non-null  object        
 2   away_team                      23921 non-null  object        
 3   home_team_continent            23921 non-null  object        
 4   away_team_continent            23921 non-null  object        
 5   home_team_fifa_rank            23921 non-null  int64         
 6   away_team_fifa_rank            23921 non-null  int64         
 7   home_team_total_fifa_points    23921 non-null  int64         
 8   away_team_total_fifa_points    23921 non-null  int64         
 9   home_team_score                23921 non-null  int64         
 10  away_team_score                23921 non-null  int64         
 11  tournament     

In [63]:
# Only look at recent games
recent = '2021-01-01'
cur_data = data[data['date']>recent]
# Remove date column
data.drop(['date'], axis = 1)
cur_data.head()

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,shoot_out,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score
22273,2021-01-12,United Arab Emirates,Iraq,Asia,Asia,74,69,1326,1347,0,...,No,Draw,71.000000,86.490909,65.200000,86.392593,68.000000,86.086364,86.700909,85.465455
22274,2021-01-18,Kuwait,Palestine,Asia,Asia,148,102,1060,1204,0,...,No,Lose,86.074074,86.490909,85.785185,86.392593,85.244444,86.086364,86.700909,85.465455
22275,2021-01-19,Dominican Republic,Puerto Rico,North America,North America,159,179,1019,941,0,...,No,Lose,86.074074,86.490909,85.785185,86.392593,85.244444,86.086364,86.700909,85.465455
22276,2021-01-22,Guatemala,Puerto Rico,North America,North America,131,179,1111,941,1,...,No,Win,86.074074,86.490909,85.785185,86.392593,85.244444,86.086364,86.700909,85.465455
22277,2021-01-25,Dominican Republic,Serbia,North America,Europe,159,30,1019,1495,0,...,No,Draw,86.074074,81.000000,85.785185,86.392593,85.244444,79.000000,80.000000,82.000000


In [64]:
# Find data relating to remaining teams
quarters_data = cur_data[cur_data['home_team'].isin(quarters_teams)]
quarters_data.head()

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,shoot_out,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score
22298,2021-03-24,Portugal,Azerbaijan,Europe,Europe,5,108,1662,1180,1,...,No,Win,84.0,86.490909,83.8,84.3,85.2,86.086364,86.700909,61.200000
22300,2021-03-24,France,Ukraine,Europe,Europe,2,24,1755,1521,1,...,No,Draw,87.0,76.000000,85.5,86.7,87.0,76.000000,77.000000,79.800000
22373,2021-03-27,Netherlands,Latvia,Europe,Europe,14,136,1609,1082,2,...,No,Win,81.0,70.000000,85.2,82.3,84.0,86.086364,86.700909,85.465455
22376,2021-03-27,Croatia,Cyprus,Europe,Europe,11,100,1617,1224,1,...,No,Win,80.0,86.490909,78.8,78.7,84.0,86.086364,86.700909,60.000000
22439,2021-03-30,Croatia,Malta,Europe,Europe,11,176,1617,951,3,...,No,Win,80.0,86.490909,78.8,78.7,84.0,86.086364,86.700909,85.465455


In [83]:
# Drop games that tied
binary_data = cur_data[cur_data['home_team_result'].isin(['Win', 'Lose'])]
binary_data.head()

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,shoot_out,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score
22274,2021-01-18,Kuwait,Palestine,Asia,Asia,148,102,1060,1204,0,...,No,Lose,86.074074,86.490909,85.785185,86.392593,85.244444,86.086364,86.700909,85.465455
22275,2021-01-19,Dominican Republic,Puerto Rico,North America,North America,159,179,1019,941,0,...,No,Lose,86.074074,86.490909,85.785185,86.392593,85.244444,86.086364,86.700909,85.465455
22276,2021-01-22,Guatemala,Puerto Rico,North America,North America,131,179,1111,941,1,...,No,Win,86.074074,86.490909,85.785185,86.392593,85.244444,86.086364,86.700909,85.465455
22278,2021-01-27,Iraq,Kuwait,Asia,Asia,69,148,1347,1060,2,...,No,Win,86.074074,86.490909,85.785185,86.392593,85.244444,86.086364,86.700909,85.465455
22280,2021-01-31,USA,Trinidad and Tobago,North America,South America,22,103,1545,1201,7,...,No,Win,76.000000,52.000000,75.500000,76.700000,75.200000,86.086364,86.700909,85.465455


In [84]:
# Create win column
def encode_win(x):
    if x == 'Win':
        return 1
    elif x == 'Draw':
        return -1
    else:
        return 0
win_col = binary_data['home_team_result'].transform(encode_win)
win_col

22274    0
22275    0
22276    1
22278    1
22280    1
        ..
23916    1
23917    0
23918    0
23919    0
23920    1
Name: home_team_result, Length: 1296, dtype: int64

In [85]:
binary_data['result'] = win_col

In [86]:
binary_data.head()

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score,result
22274,2021-01-18,Kuwait,Palestine,Asia,Asia,148,102,1060,1204,0,...,Lose,86.074074,86.490909,85.785185,86.392593,85.244444,86.086364,86.700909,85.465455,0
22275,2021-01-19,Dominican Republic,Puerto Rico,North America,North America,159,179,1019,941,0,...,Lose,86.074074,86.490909,85.785185,86.392593,85.244444,86.086364,86.700909,85.465455,0
22276,2021-01-22,Guatemala,Puerto Rico,North America,North America,131,179,1111,941,1,...,Win,86.074074,86.490909,85.785185,86.392593,85.244444,86.086364,86.700909,85.465455,1
22278,2021-01-27,Iraq,Kuwait,Asia,Asia,69,148,1347,1060,2,...,Win,86.074074,86.490909,85.785185,86.392593,85.244444,86.086364,86.700909,85.465455,1
22280,2021-01-31,USA,Trinidad and Tobago,North America,South America,22,103,1545,1201,7,...,Win,76.000000,52.000000,75.500000,76.700000,75.200000,86.086364,86.700909,85.465455,1
